In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:99% !important; }</style>")) # To make the cells wider

# Data Creation

In [2]:
import numpy as np
import pandas as pd
import string
import math

np.random.seed(2) # set a random seed so that the numbers don't change every time I run this cell
digits = [*string.digits]
letters = [*string.ascii_uppercase]
product_id_list = {''.join(np.random.choice(digits, 3))+'-'+''.\
                   join(np.random.choice(letters, 3))+'-'+''.\
                   join(np.random.choice(digits, 4)) for i in range(500)} # make it a set so that all product IDs must be unique

category_list = np.random.choice(['Top','Bottom','Underwear','Headwear'], size=len(product_id_list), p=[0.4,0.35,0.2,0.05])
gender_list = np.random.choice(['Male','Female','Gender neutral'], size=len(product_id_list), p=[0.47,0.47,0.06])
usa_revenue = pd.Series(np.random.lognormal(mean=10, sigma=0.6, size=len(product_id_list))).round(2)
canada_revenue = [round(np.random.lognormal(-1.5, 0.4)*i, 2) for i in usa_revenue.values]
china_revenue = [round(np.random.lognormal(-0.1, 0.33)*i, 2) for i in usa_revenue.values]
india_revenue = [round(np.random.lognormal(-0.4, 0.25)*i, 2) for i in usa_revenue.values]

df = pd.DataFrame({'SKU':list(product_id_list), 'prod_category':category_list, 'prod_gender':gender_list,
                   'usa_revenue':usa_revenue, 'canada_revenue':canada_revenue, 'china_revenue':china_revenue,
                   'india_revenue':india_revenue}) # Creating the DataFrame with categories and revenue data
for i in ['usa','canada','china','india']: # Adding unit sales data
    df[i+'_unit_sales'] = [math.ceil(val/np.random.lognormal(mean=3, sigma=0.2)) for val in df[i+'_revenue'].values]
df

,SKU,prod_category,prod_gender,usa_revenue,canada_revenue,china_revenue,india_revenue,usa_unit_sales,canada_unit_sales,china_unit_sales,india_unit_sales
0,828-SZS-9696,Top,Gender neutral,12800.46,2637.88,6202.08,7632.09,671,128,332,392
1,235-KSY-7655,Top,Female,27082.90,2801.88,31117.55,26319.45,1136,118,1448,1361
2,007-NIS-2249,Bottom,Female,52195.56,12083.77,49963.14,42676.40,2782,600,2683,2372
3,431-JMJ-9525,Underwear,Male,17302.96,5435.48,8136.07,12478.48,812,304,476,583
4,088-ZTN-5009,Bottom,Female,21857.09,3873.83,15992.00,18811.36,991,232,780,947
...,...,...,...,...,...,...,...,...,...,...,...
495,626-XVQ-1569,Bottom,Female,48311.55,7956.22,30575.67,23418.59,2022,552,1494,1286
496,394-NIE-0047,Bottom,Male,22894.49,3892.61,24057.84,15928.80,1084,243,1631,721
497,123-UAY-9485,Bottom,Male,29454.55,5387.46,14140.66,25311.49,1679,206,818,1362
498,817-NZW-1741,Top,Male,20450.75,8430.02,26027.92,12582.68,1305,298,1712,451


# Question

This is a sample data set from an imaginary apparel company that sells its products in Canada, China, India, and the US. Reshape the DataFrame <i>df</i> (printed in the <i>Data Creation</i> section above) into a new DataFrame (preserving all of the initial data) with the following columns: <b>"SKU", "prod_category", "prod_gender", "country", "revenue", "unit_sales"</b>. A sample of the correct final output is shown below.
<img src='solution_sample.png' style='width:550px;'/>


# Solution

In [14]:
rev_df = pd.melt(df, id_vars=["SKU", "prod_category", "prod_gender"], value_vars=df.columns[3:7], var_name='country', value_name='revenue')
sales_df = pd.melt(df, id_vars=["SKU", "prod_category", "prod_gender"], value_vars=df.columns[7:], var_name='country', value_name='unit_sales')

for data in [rev_df, sales_df]:
    data['country'] = data['country'].map(lambda x: x.split('_')[0].title().replace('Usa','USA'))

In [18]:
tidy_df = pd.merge(rev_df, sales_df, on=["SKU", "prod_category", "prod_gender", "country"])
tidy_df

,SKU,prod_category,prod_gender,country,revenue,unit_sales
0,828-SZS-9696,Top,Gender neutral,USA,12800.46,671
1,235-KSY-7655,Top,Female,USA,27082.90,1136
2,007-NIS-2249,Bottom,Female,USA,52195.56,2782
3,431-JMJ-9525,Underwear,Male,USA,17302.96,812
4,088-ZTN-5009,Bottom,Female,USA,21857.09,991
...,...,...,...,...,...,...
1995,626-XVQ-1569,Bottom,Female,India,23418.59,1286
1996,394-NIE-0047,Bottom,Male,India,15928.80,721
1997,123-UAY-9485,Bottom,Male,India,25311.49,1362
1998,817-NZW-1741,Top,Male,India,12582.68,451
